### 01 packages

In [1]:
##########################################################################################
##########################################################################################

import matplotlib.pyplot as plt

from matplotlib.ticker import MultipleLocator, FormatStrFormatter

##########################################################################################
##########################################################################################

import numpy as np

import pandas as pd

##########################################################################################
##########################################################################################

import os,sys

import copy

import math

import random

import time

import datetime

##########################################################################################
##########################################################################################

import h3

import folium

from shapely.geometry import Point, Polygon

import geopandas as gp

import networkx as nx

##########################################################################################
##########################################################################################

import warnings

warnings.filterwarnings("ignore")

### 02 Manhattan Network

In [2]:
##########################################################################################
##########################################################################################

latitude = 40.7831

longitude = -73.9712

##########################################################################################
##########################################################################################

def Get_POLYGON(coords):
    
    if coords.type=='Polygon':
        
        return coords
    
    else:
        
        Score={i:coords[i].area for i in range(len(coords))}
        
        idx=max(Score, key=Score.get)
        
        return coords[idx]
    
##########################################################################################
##########################################################################################

Zone_data = gp.read_file('./NYC_Zones/geo_export_789b911c-2a6d-4092-a682-6c5e6ce53409.shp')

Zone_data=Zone_data[['borough','zone','geometry','location_i']]

Zone_data=Zone_data.loc[Zone_data['borough']=='Manhattan']

Zone_data=Zone_data.reset_index(drop=True)

##########################################################################################
##########################################################################################

Zone_data['Zone_id']=['Zone_'+str(i) for i in Zone_data.index]

Zone_data=Zone_data[['Zone_id','borough','zone','geometry','location_i']]

Islands=['Zone_19','Zone_20','Zone_21','Zone_38','Zone_47','Zone_48']

Zone_data=Zone_data.loc[~Zone_data['Zone_id'].isin(Islands)]

Zone_data=Zone_data.reset_index(drop=True)

##########################################################################################
##########################################################################################

Zone_data['Zone_id']=['Zone_'+str(i) for i in Zone_data.index]

Zone_data['geometry']=Zone_data.apply(lambda x:Get_POLYGON(x['geometry']),axis=1)

Zone_data['center'] = Zone_data.apply(lambda x:x['geometry'].centroid,axis=1)

Zone_data=Zone_data.rename(columns={'location_i':'LocationID'})

Zone_data[['LocationID']] = Zone_data[['LocationID']].astype(float)

Zone_data['type']=Zone_data.apply(lambda x:x['geometry'].type,axis=1)

##########################################################################################
##########################################################################################

Zone_geometry={}

for idx,row in Zone_data.iterrows():
    
    Zone_geometry[int(row['LocationID'])]=row['geometry']

Zone_geometry

np.save('./Processed_data/Zone_geometry',Zone_geometry)

##########################################################################################
##########################################################################################

network = folium.Map(location=[latitude, longitude], zoom_start=12,tiles='CartoDB positron')

##########################################################################################
##########################################################################################

folium.Choropleth(
    geo_data=Zone_data[['geometry']],
    fill_color='blue',
    fill_opacity=0.1,
    name='Zone').add_to(network)

network


### 03 Hexagon network with lockers

In [3]:
Hexagons=list()

##########################################################################################
##########################################################################################

# Function to calculate the midpoint between two coordinates

def calculate_midpoint(coord1, coord2):
    
    return ((coord1[0] + coord2[0]) / 2, (coord1[1] + coord2[1]) / 2)

##########################################################################################
##########################################################################################

# The desired resolution for the H3 hexagon

resolution = 7

# Number of rings away from the original hexagon to get neighbors

num_rings_away = 20

##########################################################################################
##########################################################################################

# Generate the H3 hexagon at the specified location and resolution

h3_hexagon = h3.geo_to_h3(latitude, longitude, resolution)


# Get the neighbors of the hexagon

neighbors = h3.k_ring(h3_hexagon, num_rings_away)

##########################################################################################
##########################################################################################

midpoints = []

hexagons_midpoints={}

midpoints_hexagons={}

##########################################################################################
##########################################################################################

for hexagon in neighbors:

    # Get the center coordinates of the hexagon
    
    hexagon_vertice=h3.h3_to_geo_boundary(hexagon)
    
    hexagon_midpoints=list()
    
    for i in range(len(hexagon_vertice)):
        
        # Calculate the midpoint between the current vertex and the next vertex (to close the polygon)
        
        midpoint = calculate_midpoint(hexagon_vertice[i], hexagon_vertice[(i + 1) % len(hexagon_vertice)])
        
        hexagon_midpoints.append(midpoint)
    
    cover_count=0
    
    for polygon in Zone_geometry.values():
    
        # Check if the coordinate is covered by the polygon
        
        for midpoint in hexagon_midpoints:
            
            midpoint_obj=list(midpoint)[::-1]

            is_covered = polygon.contains(Point(midpoint_obj))

            cover_count+=is_covered
            
            if cover_count>=1:

                Hexagons.append(hexagon)
                
                midpoints+=hexagon_midpoints
                
                midpoints=list(set(midpoints))
                
                hexagons_midpoints[hexagon]=hexagon_midpoints
                
                ##########################################################################################
                ##########################################################################################
                
                for point in hexagon_midpoints:
                
                    if point not in midpoints_hexagons.keys():

                        midpoints_hexagons[point]=[hexagon]

                    else:
                        
                        if hexagon not in midpoints_hexagons[point]:

                            midpoints_hexagons[point].append(hexagon)


                break
                
##########################################################################################
##########################################################################################

Hexagons=list(set(Hexagons))

##########################################################################################
##########################################################################################

Lockers=list()

for midpoint in midpoints:
    
     for polygon in Zone_geometry.values():
            
        is_covered = polygon.contains(Point(midpoint[::-1]))
            
        if is_covered and len(midpoints_hexagons[midpoint])==2:
            
            Lockers.append(midpoint)

##########################################################################################
##########################################################################################

Hexagons_Lockers={}

for hexagon in Hexagons:
    
    Hexagons_Lockers[hexagon]=[locker for locker in hexagons_midpoints[hexagon] if locker in Lockers]

##########################################################################################
##########################################################################################

Lockers_Hexagons={}

for locker in Lockers:
    
    Lockers_Hexagons[locker]=[hexagon for hexagon in midpoints_hexagons[locker] if hexagon in Hexagons]

##########################################################################################
##########################################################################################

np.save('./Processed_data/Hexagons',Hexagons)

np.save('./Processed_data/Lockers',Lockers)

np.save('./Processed_data/Hexagons_Lockers',Hexagons_Lockers)

np.save('./Processed_data/Lockers_Hexagons',Lockers_Hexagons)

##########################################################################################
##########################################################################################


network = folium.Map(location=[latitude, longitude], zoom_start=12,tiles='CartoDB positron')

hexagon_vertices = {hexagon:h3.h3_to_geo_boundary(hexagon) for hexagon in Hexagons}

for vertices in hexagon_vertices.values():
    
    folium.Polygon(locations=vertices, color='black', opacity=1, fill=False, fill_color='blue', fill_opacity=0.0).add_to(network)
    
for midpoint in Lockers:

    folium.Circle(
    radius=20,
    location=midpoint,
    color='red',
    fill=True,
    ).add_to(network)

    
network


### 03 Locker network

In [4]:
##########################################################################################
##########################################################################################


def haversine_distance(lat1, lon1, lat2, lon2):
    
    return Point(lat1, lon1).distance(Point(lat2, lon2))*111194.92


##########################################################################################
##########################################################################################

Locaker_G = nx.Graph()

Locaker_G.add_nodes_from(Lockers)

for locker in Lockers:
    
    for hexagon in Lockers_Hexagons[locker]:
        
        for locker_ in Hexagons_Lockers[hexagon]:
            
            if locker_ != locker:
                
                distance=haversine_distance(locker[0],locker[1],locker_[0],locker_[1])
                
                Locaker_G.add_edge(locker, locker_,weight=distance)
    
nx.write_graphml(Locaker_G, "./Processed_data/Locaker_G.graphml")

##########################################################################################
##########################################################################################

network = folium.Map(location=[latitude, longitude], zoom_start=12,tiles='CartoDB positron')

hexagon_vertices = {hexagon:h3.h3_to_geo_boundary(str(hexagon)) for hexagon in Hexagons}

for hexagon,vertices in hexagon_vertices.items():
    
    folium.Polygon(locations=vertices, 
                   color='black', 
                   opacity=1, 
                   fill=False,
                   weight=1,).add_to(network)
    
# Convert NetworkX graph to a GeoDataFrame for Folium
graph_geo = nx.to_pandas_edgelist(Locaker_G, source='source', target='target')

# Add graph to Folium map as a PolyLine

for index, row in graph_geo.iterrows():
    
    folium.PolyLine([row['source'], row['target']], color='blue',weight=1).add_to(network)

for midpoint in Lockers:
    
    locker=(midpoint[0], midpoint[1])
    
    if len(Lockers_Hexagons[locker])==2:
    
        folium.Circle(
        radius=10,
        location=locker,
        color='red',
        fill=True,
        ).add_to(network)
        
network.save('./Html/01_Locker_nework.html')
    
network

### Hexagon network with grocerys

In [5]:
##########################################################################################
##########################################################################################

files=[file for file in os.listdir("./Grocery/") if '.xlsx' in file]

files=sorted(files)

##########################################################################################
##########################################################################################

grocery_df=pd.DataFrame([],columns=['name','latitude', 'longitude'])

for file in files:
    
    df = pd.read_excel('./Grocery/'+file)

    df=df[['name','latitude', 'longitude']]

    df=df.dropna()
    
    grocery_df= pd.concat([grocery_df, df], ignore_index=True)

grocery_df=grocery_df.drop_duplicates(subset=['name'])

grocery_df=grocery_df.reset_index(drop=True)

grocery_df.to_csv('./Grocery/grocery_df.csv')

##########################################################################################
##########################################################################################

grocerys=list()

for idx,row in grocery_df.iterrows():
    
    grocery_location=Point(row.longitude,row.latitude)
    
    for polygon in Zone_geometry.values():
    
        # Check if the coordinate is covered by the polygon

        is_covered = polygon.contains(grocery_location)
        
        grocery_hexagon=h3.geo_to_h3(row.latitude, row.longitude,resolution)
        
        if is_covered and grocery_hexagon in Hexagons:
            
            grocerys.append((row.longitude,row.latitude))
            
            break
            
np.save('./Processed_data/Grocerys',grocerys)

##########################################################################################
##########################################################################################

network = folium.Map(location=[latitude, longitude], zoom_start=12,tiles='CartoDB positron')

for vertices in hexagon_vertices.values():
    
    folium.Polygon(locations=vertices, 
                   color='black', 
                   opacity=1, 
                   fill=False,
                   weight=1,).add_to(network)
for locker in Lockers:
    
    folium.Circle(
    radius=10,
    location=locker,
    color='red',
    fill=True,
    ).add_to(network)
    
for grocery in grocerys:
    
    folium.Circle(
    radius=20,
    location=grocery[::-1],
    color='green',
    fill=True,
    ).add_to(network)


network.save('./Html/02_Hexagon_with_grocery.html')
    
network